In [1]:
import joblib
from sklearn.model_selection import train_test_split

# 이전 아마존 데이터 로더
with open('amazon.pkl', 'rb') as f:
    data = joblib.load(f)
locals().update(data)

#tdm 을 확인해보자.
tdm = tdm.toarray()
print(tdm)


# 트레인,테스트 데이터 분리 
X_train, X_test, y_train, y_test = train_test_split(tdm, sentiment, test_size=.2, random_state=1234)

# 케라스 입력 개수 설정 : tfidf 단어 갯수 
NUM_WORDS = X_train.shape[1]

print(NUM_WORDS)


[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
1000


In [2]:
# 1. 케라스 활용 로지스틱 회귀 분석
# - 모형 만들기

from keras import Sequential
from keras.layers import Dense, Input

# 단어의 갯수(NUM_WORDS)만큼 입력을 받아 1개의 출력을 내놓는 레이어를 추가한다. Dense는 선형 모형, sigmoid는 로지스틱 함수와 같다.
# 입력과 동시에 출력 모드 

m1 = Sequential()
m1.add(Dense(1, activation='sigmoid', input_shape=(NUM_WORDS,)))
m1.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1)                 1001      
                                                                 
Total params: 1001 (3.91 KB)
Trainable params: 1001 (3.91 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [3]:
from keras.optimizers import Adam

# 손실 함수는 크로스엔트로피를 쓴다. 
# 문제에 따라 크로스엔트로피를 효과적으로 계산하는 방법이 달라지는데 둘 중에 하나로 분류할 때는 binary_crossentropy를 사용한다. 
# 3개 이상의 분류에는 categorical_crossentropy를 사용한다. 
# 최적화 알고리즘으로는 Adam을 사용한다.

m1.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer=Adam())
m1.fit(X_train, y_train, epochs=30)


Epoch 1/30
25/25 [==============================] - 3s 10ms/step - loss: 0.6921 - accuracy: 0.5238
Epoch 2/30
25/25 [==============================] - 0s 6ms/step - loss: 0.6870 - accuracy: 0.5875
Epoch 3/30
25/25 [==============================] - 0s 4ms/step - loss: 0.6821 - accuracy: 0.6787
Epoch 4/30
25/25 [==============================] - 0s 4ms/step - loss: 0.6773 - accuracy: 0.7412
Epoch 5/30
25/25 [==============================] - 0s 4ms/step - loss: 0.6726 - accuracy: 0.7750
Epoch 6/30
25/25 [==============================] - 0s 4ms/step - loss: 0.6678 - accuracy: 0.8037
Epoch 7/30
25/25 [==============================] - 0s 5ms/step - loss: 0.6632 - accuracy: 0.8275
Epoch 8/30
25/25 [==============================] - 0s 5ms/step - loss: 0.6586 - accuracy: 0.8450
Epoch 9/30
25/25 [==============================] - 0s 4ms/step - loss: 0.6541 - accuracy: 0.8612
Epoch 10/30
25/25 [==============================] - 0s 4ms/step - loss: 0.6496 - accuracy: 0.8687
Epoch 11/30
25/25 

In [4]:
# - 예측
# y_pred = m1.predict_classes(X_test)  keras 2.3 부터 변경됨 
from sklearn.metrics import accuracy_score

y_pred = m1.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)
#print(y_pred)
#print(y_test)
#print(y_pred_classes)

# 모델 정확도 계산 . 이전 실행한 사이킷런의 로지스틱 회귀랑 비교
accuracy_score(y_test, y_pred)  


7/7 [==============================] - 0s 7ms/step


0.735

In [5]:
# 다층 신경망 분석 

#  - 신경망 생성 및 학습
m2 = Sequential()
m2.add(Dense(100, activation='relu', input_shape=(NUM_WORDS,)))
# Dense 추가 
m2.add(Dense(50, activation='relu', input_shape=(100,)))
m2.add(Dense(100, activation='relu', input_shape=(50,)))
# Dense 추가 
m2.add(Dense(1, activation='sigmoid'))
m2.summary()   # 모델 확인 


m2.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer=Adam())
m2.fit(X_train, y_train, epochs=30,verbose=0)



Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 100)               100100    
                                                                 
 dense_2 (Dense)             (None, 50)                5050      
                                                                 
 dense_3 (Dense)             (None, 100)               5100      
                                                                 
 dense_4 (Dense)             (None, 1)                 101       
                                                                 
Total params: 110351 (431.06 KB)
Trainable params: 110351 (431.06 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [6]:
# y_pred2 = m2.predict_classes(X_test)

# 모델 정확도 계산 . 이전 실행한 사이킷런의 로지스틱 회귀랑 비교

y_pred2 = m2.predict(X_test)
y_pred2 = (y_pred2 > 0.5).astype(int)

accuracy_score(y_test, y_pred2)


7/7 [==============================] - 0s 7ms/step


0.81